#**EDA**

###**Instalaciones y Herramientas del Proyecto**




In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.2 MB/s eta 0:00:00


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.7 MB/s eta 0:00:00


##**Librerias**

In [ ]:
#Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from category_encoders import CatBoostEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import optuna
pd.set_option('display.max_columns', None)

##**Funciones**

In [ ]:
#Funcion resumen de datos
def resumen(dataframe):
  # Cálculos de resumen
  cantidad_nulos= dataframe.isnull().sum()
  cantidad_total = len(dataframe)
  cantidad_no_nulos = dataframe.notnull().sum()
  tipo_de_datos = dataframe.dtypes
  cantidad_de_clases = dataframe.nunique()
  # Crear DataFrame de resumen
  resumen_df = pd.DataFrame({
      'Tipo de Dato': tipo_de_datos,
      'Valores Únicos': cantidad_de_clases,
      'No Nulos': cantidad_no_nulos,
      'Nulos': cantidad_nulos,
      '% Nulos': (cantidad_nulos / cantidad_total) * 100
  })
  # Ordenar por % de nulos
  resumen_df = resumen_df.sort_values(by='% Nulos', ascending=False)
  # Aplicar formato con Styler
  tabla_resumen = resumen_df.style \
      .format({'% Nulos': '{:.2f}%'}) \
      .bar(subset='% Nulos', color='#f08080') \
      .set_properties(**{'text-align': 'center'}) \
      .set_caption("Resumen de Columnas del DataFrame") \
      .set_table_styles([
          # Color del encabezado de columnas
          {'selector': 'th', 'props': [
              ('text-align', 'center'),
              ('background-color', '#1f4e79'),
              ('color', 'white'),
              ('font-weight', 'bold')
          ]},
          # Color de las celdas
          {'selector': 'td', 'props': [('padding', '5px')]},
          # Estilo del título de la tabla
          {'selector': 'caption', 'props': [
              ('color', '#1f4e79'),
              ('font-size', '18px'),
              ('text-align', 'center'),
              ('font-weight', 'bold')
          ]}
      ])
  # Mostrar
  display(tabla_resumen)
  # Información final
  print(f"\n🧾 El DataFrame tiene un total de **{len(dataframe.columns)}** columnas y **{len(dataframe)}** filas.")

In [ ]:
#Función Encabezado
def encabezado(dataframe,filas=5):
  salida=dataframe.head(filas).style \
      .set_properties(**{
          'background-color': '#f9f9f9',
          'border': '1px solid #ddd',
          'text-align': 'center'
      }) \
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', '#4B8BBE'), ('color', 'white'), ('text-align', 'center')]}
      ]) \
      .highlight_null() \
      .set_caption("🔍 Vista Previa de los Primeros Registros")
  display(salida)

In [ ]:
#Chi cuadrado y cramer
def chi2_y_cramers_v(df, var1, var2):
    tabla = pd.crosstab(df[var1], df[var2])
    chi2, p, dof, expected = chi2_contingency(tabla)

    n = tabla.sum().sum()
    r, k = tabla.shape
    v_cramer = np.sqrt(chi2 / (n * (min(r, k) - 1)))

    print(f"{var1}")
    print(f"Estadístico chi² = {chi2:.2f}")
    print(f"p-valor = {p:.4f}")
    print(f"V de Cramer = {v_cramer:.4f}")

In [ ]:
#Funcion de Agrupamiento DX
def agrupar_dx(data, columna):
  nueva_llista_dx_egreso = []
  for i in data[columna].tolist():
          if not isinstance(i, str) or len(i) < 3:
              nueva_llista_dx_egreso.append('OTROS')
              continue
          letra = i[0].upper()
          try:
              num = int(i[1:3])
          except:
              num = None

          if letra in ['A', 'B']:
              nueva_llista_dx_egreso.append('A-B')
          elif letra == 'C':
              nueva_llista_dx_egreso.append('C-D1')
          elif letra == 'D':
              if num is not None and num <= 48:
                  nueva_llista_dx_egreso.append('C-D2')
              else:
                  nueva_llista_dx_egreso.append('D-D')
          elif letra == 'E':
              nueva_llista_dx_egreso.append('E-E')
          elif letra == 'F':
              nueva_llista_dx_egreso.append('F-F')
          elif letra == 'G':
              nueva_llista_dx_egreso.append('G-G')
          elif letra == 'H':
              nueva_llista_dx_egreso.append('H-H')
          elif letra == 'I':
              if num is not None and (
                  10 <= num <= 15 or
                  20 <= num <= 25 or
                  60 <= num <= 69
              ):
                  nueva_llista_dx_egreso.append('I1')  # Enfermedades cardiovasculares más comunes
              else:
                  nueva_llista_dx_egreso.append('I2')  # Otras enfermedades del sistema circulatorio
          elif letra == 'J':
              nueva_llista_dx_egreso.append('J-J')
          elif letra == 'K':
              if num is not None and 0 <= num <= 63:
                  nueva_llista_dx_egreso.append('K1')  # Digestivo general
              else:
                  nueva_llista_dx_egreso.append('K2')  # Hepatobiliar, etc.
          elif letra == 'L':
              nueva_llista_dx_egreso.append('L-L')
          elif letra == 'M':
              nueva_llista_dx_egreso.append('M-M')
          elif letra == 'N':
              nueva_llista_dx_egreso.append('N-N')
          elif letra in ['O', 'P', 'Q', 'R', 'Z']:
              nueva_llista_dx_egreso.append('P-Q-O-R-Z')
          elif letra == 'S':
              nueva_llista_dx_egreso.append('S-T1')
          elif letra == 'T':
              nueva_llista_dx_egreso.append('S-T2')
          elif letra in ['V', 'W', 'X', 'Y']:
              nueva_llista_dx_egreso.append('S-T3')  # Puedes cambiar este nombre si lo prefieres
          else:
              nueva_llista_dx_egreso.append('OTROS')
  data[columna] = nueva_llista_dx_egreso
  reagrupacion = {
    'J-J': 'Respiratorio',
    'N-N': 'Genitourinario',
    'I1': 'Circulatorio',
    'I2': 'Circulatorio',
    'S-T1': 'Traumatismo',
    'S-T2': 'Traumatismo',
    'S-T3': 'Traumatismo',
    'C-D1': 'Neoplasias',
    'C-D2': 'Neoplasias',
    'D-D': 'Neoplasias',
    'A-B': 'Infecciosas',
    'K1': 'Digestivo',
    'K2': 'Digestivo',
    'G-G': 'Neurológico',
    'F-F': 'Psicológico',
    'E-E': 'Endocrino',
    'M-M': 'Musculoesquelético',
    'L-L': 'Dermatológico',
    'H-H': 'Sensorial',
    'P-Q-O-R-Z': 'Otros',
    'OTROS':'Vacio'
}
  data[columna] = data[columna].map(reagrupacion)

In [ ]:
#Agrupacion de los procedimientos:
def agrupar_cie9_mc(codigo):
    try:
        cod = float(codigo)
        if 0.00 <= cod <= 0.99:
            return 'Misceláneos'
        elif 1.00 <= cod <= 5.99:
            return 'Sistema nervioso'
        elif 6.00 <= cod <= 7.99:
            return 'Endócrino'
        elif 8.00 <= cod <= 16.99:
            return 'Ojo y oído'
        elif 17.00 <= cod <= 20.99:
            return 'Respiratorio'
        elif 21.00 <= cod <= 29.99:
            return 'Cardiovascular'
        elif 30.00 <= cod <= 34.99:
            return 'Linfático y bazo'
        elif 35.00 <= cod <= 39.99:
            return 'Digestivo'
        elif 40.00 <= cod <= 54.99:
            return 'Genitourinario'
        elif 55.00 <= cod <= 59.99:
            return 'Parto y puerperio'
        elif 60.00 <= cod <= 64.99:
            return 'Musculoesquelético'
        elif 65.00 <= cod <= 71.99:
            return 'Tegumentario'
        elif 72.00 <= cod <= 75.99:
            return 'Obstetricia'
        elif 76.00 <= cod <= 84.99:
            return 'Cirugía músculo-esquelética'
        elif 85.00 <= cod <= 86.99:
            return 'Mama y piel'
        elif 87.00 <= cod <= 99.99:
            return 'Diagnóstico/terapéutico'
        else:
            return 'Otros'
    except:
        return 'Desconocido'

##**Visualizacion General de los Datos**

In [ ]:
#Lectura del documento compartido
data = pd.read_excel('Base_Datos.xlsx')

In [ ]:
resumen(data)

,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
Dxr 10,object,1,1,27868,100.00%
Proc30,float64,5,5,27864,99.98%
Proc29,float64,6,9,27860,99.97%
Proc28,float64,10,15,27854,99.95%
Proc27,float64,13,20,27849,99.93%
Proc26,float64,16,23,27846,99.92%
Proc25,float64,16,26,27843,99.91%
Proc24,float64,22,31,27838,99.89%
Proc23,float64,20,39,27830,99.86%
Proc22,float64,28,49,27820,99.82%



🧾 El DataFrame tiene un total de **68** columnas y **27869** filas.


In [ ]:
encabezado(data)

,Num caso,Aseguradora -Código-,Aseguradora -Descripción-,Edad,Grupo Edad,Sexo,Codigo de ciudad,Fecha de ingreso,Tipo de ingreso,Fecha de egreso,Días estancia,ServicioAlta,Cuidados intensivos,Días de Unidad Cuidado Intensivo,Dx de ingreso,Dx principal de egreso,Dx principal de egreso .1,Dx Ppal 3 Caracteres,Dxr 1,Dxr 2,Dxr 3,Dxr 4,Dxr 5,Dxr-6,Dxr 7,Dxr 8,Dxr 9,Dxr 10,Código causa externa,Causa externa,Situacion al alta,Proc1,Proc2,Proc3,Proc4,Proc5,Proc6,Proc7,Proc8,Proc9,Proc10,Proc11,Proc12,Proc13,Proc14,Proc15,Proc16,Proc17,Proc18,Proc19,Proc20,Proc21,Proc22,Proc23,Proc24,Proc25,Proc26,Proc27,Proc28,Proc29,Proc30,Tipo servicio,Causa Basica de muerte,Infecciones,Infección Quirurgica,GRD -Código,GRD -Descripción,Tipo GRD
0,413799.000000,204,Aseguradora_29,38,35-39,F,5001,30052005,URGENCIA,19062005,20,29,NO,0,2,I635,I63,INFARTO CEREBRAL,I652,C189,C787,C788,N390,A048,D648,R060,nan,nan,nan,nan,ALTA MEDICA,43.110000,54.910000,87.440000,87.030000,88.720000,88.760000,88.710000,89.510000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,URGENCIA ADULTOS,nan,NO,NO,14143,MH ACCIDENTE VASCULAR CEREBRAL CON INFARTO w/MCC,M
1,412359.000000,313,Aseguradora_56,27,25-29,M,5088,11032005,URGENCIA,10042005,30,156,NO,0,2,B232,B23,"ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA [ VIH ], RESULTANTE EN OTRAS AFECCIONES",B210,B204,B162,A530,D538,A071,E538,D70,A071,nan,nan,nan,ALTA MÉDICA,87.440000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NO APLICA,nan,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HUMANA w/MCC,M
2,412552.000000,313,Aseguradora_56,39,35-39,M,5001,20032005,URGENCIA,10042005,21,39,NO,0,2,B227,B22,"ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA [ VIH ], RESULTANTE EN OTRAS ENFERMEDADES ESPECIFICADAS",B451,G052,B588,D70,D698,A072,D618,E871,E876,nan,nan,nan,ALTA MÉDICA,38.930000,45.230000,87.440000,87.410000,88.010000,88.760000,88.970000,88.970000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,URGENCIA ADULTOS,nan,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HUMANA w/MCC,M
3,414131.000000,227,Aseguradora_46,24,20-24,M,5001,20062005,URGENCIA,22062005,2,20,NO,0,2,B208,B20,"ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA [ VIH ], RESULTANTE EN ENFERMEDADES INFECCIOSAS Y PARASITARIAS",B200,B210,A600,N518,A530,A073,D696,R103,nan,nan,nan,nan,ALTA MÉDICA,88.010000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,URGENCIA ADULTOS,nan,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HUMANA w/MCC,M
4,410573.000000,227,Aseguradora_46,60,60-64,M,5001,24112004,URGENCIA,7012005,44,24,NO,0,2,T814,T81,COMPLICACIONES DE PROCEDIMIENTOS NO CLASIFICADAS EN OTRA PARTE,Y838,C140,N390,B961,B962,A099,D508,E871,Z931,nan,Y838,Otros Procedimientos Quirurgicos,ALTA MÉDICA,45.160000,87.440000,88.790000,88.760000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,URGENCIA ADULTOS,nan,NO,NO,184112,MH INFECCIONES POST OPERATORIAS Y POSTRAUMÁTICAS w/CC,M


##**Analisis de Features**

###**Eliminacion e Imputacion**

**Objetivo del estudio:**

El objetivo principal de este estudio es diseñar un modelo predictivo clínico que permita clasificar adecuadamente los Grupos Relacionados por el Diagnóstico (GRD). Al tratarse de una predicción con enfoque clínico, se han excluido variables de naturaleza administrativa, como la aseguradora y la ciudad, debido al riesgo de introducir sesgos no deseados en el modelo. Asimismo, se ha descartado la variable "número de caso", por tratarse de un identificador único sin valor predictivo.

De igual manera, se ha eliminado la variable "grupo de edad" por su redundancia con la variable "edad", ya presente en el conjunto de datos y con mayor capacidad de representar diferencias individuales. Finalmente, variables como "causa de muerte" y "causa externa" han sido excluidas debido a su alto porcentaje de valores nulos y a su limitada utilidad predictiva, ya que no cumplen una doble función clínico-asistencial como sí lo hacen los diagnósticos y procedimientos.

In [ ]:
#Eliminación de parametros inecesarios
data_version_1=data.drop(columns=['Grupo Edad','Num caso','Aseguradora -Código-','Aseguradora -Descripción-','Codigo de ciudad','Causa Basica de muerte',
                                  'Código causa externa','Causa externa'])

In [ ]:
#Renombramiento de Columnas, para mejorar la legibilidad de las mismas
nuevos_nombres= ['EDAD','SEXO','FECHA_INGRESO','TIPO_INGRESO','FECHA_EGRESO','DIAS_ESTANCIA','SERVICIO_ALTA','CUIDADOS_INTENSIVOS',
'DIAS_CUIDADOS_INTENSIVOS','DX_INGRESO','DX_PRINCIPAL_EGRESO','DX_PRINCIPAL_EGRESO_1','DX_Principal_3_Caracteres','DXR_1','DXR_2',
'DXR_3','DXR_4','DXR_5','DXR_6','DXR_7','DXR_8','DXR_9','DXR_10','SITUACION_ALTA',
'PROC_1','PROC_2','PROC_3','PROC_4','PROC_5','PROC_6','PROC_7','PROC_8','PROC_9','PROC_10','PROC_11','PROC_12','PROC_13','PROC_14','PROC_15','PROC_16',
'PROC_17','PROC_18','PROC_19','PROC_20','PROC_21','PROC_22','PROC_23','PROC_24','PROC_25','PROC_26','PROC_27','PROC_28','PROC_29','PROC_30',
'TIPO_SERVICIO','INFECCIONES','INFECCION_QUIRURGICA','GRD_CODIGO','GRD_DESCRIPCION','TIPO_GRD']
data_version_1=data_version_1.set_axis(nuevos_nombres, axis=1)

In [ ]:
#Los nulos de los procedimientos medicos, como de los diagnosticos, seran tomados como no validos
#Por los cual se les llenara con una notacion especifica
for i in range(1,31):
  data_version_1[f'PROC_{i}'] = data_version_1[f'PROC_{i}'].fillna('VACIO')
for i in range(1,11):
  data_version_1[f'DXR_{i}'] = data_version_1[f'DXR_{i}'].fillna('0')

In [ ]:
resumen(data_version_1)

,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SITUACION_ALTA,object,6,27868,1,0.00%
TIPO_SERVICIO,object,13,27868,1,0.00%
FECHA_INGRESO,int64,419,27869,0,0.00%
EDAD,int64,108,27869,0,0.00%
FECHA_EGRESO,int64,365,27869,0,0.00%
DIAS_ESTANCIA,int64,107,27869,0,0.00%
SERVICIO_ALTA,int64,26,27869,0,0.00%
CUIDADOS_INTENSIVOS,object,6,27869,0,0.00%
DIAS_CUIDADOS_INTENSIVOS,int64,66,27869,0,0.00%
DX_INGRESO,int64,2,27869,0,0.00%



🧾 El DataFrame tiene un total de **60** columnas y **27869** filas.


###**Analisis Diagnosticos**

**Selección de la variable de diagnóstico de egreso:**

El diagnóstico de egreso se encuentra representado en tres columnas distintas dentro del conjunto de datos: una corresponde a la descripción textual del código, mientras que las otras dos contienen la codificación CIE-10, con distintos niveles de especificidad. Una de estas versiones presenta una codificación más general, con menos clases (aproximadamente 890), y la otra es más específica, lo que incrementa la cantidad de categorías posibles. Al analizar su contenido, se observa que representan esencialmente la misma información, por lo que conservar las tres resultaría redundante. Por tanto, se decidió eliminar dos de ellas y conservar únicamente una versión codificada, descartando también la descripción textual del diagnóstico.

In [ ]:
#DX Egreso:
print(data_version_1['DX_PRINCIPAL_EGRESO'].value_counts()[data_version_1['DX_PRINCIPAL_EGRESO'].value_counts()>120])
print(f'\ncantidad de clases menores al 10%: {len(data_version_1.DX_PRINCIPAL_EGRESO.value_counts()[data_version_1.DX_PRINCIPAL_EGRESO.value_counts()<120])}')
print(f'cantidad de clases mayores al 10%: { len(data_version_1.DX_PRINCIPAL_EGRESO.value_counts())}')
print('')
print(data_version_1['DX_PRINCIPAL_EGRESO_1'].value_counts()[data_version_1['DX_PRINCIPAL_EGRESO_1'].value_counts()>120])
print(f'\ncantidad de clases menores al 10%: {len(data_version_1.DX_PRINCIPAL_EGRESO_1.value_counts()[data_version_1.DX_PRINCIPAL_EGRESO_1.value_counts()<120])}')
print(f'cantidad de clases mayores al 10%: { len(data_version_1.DX_PRINCIPAL_EGRESO_1.value_counts())}')


DX_PRINCIPAL_EGRESO
N390     1338
J441      757
J188      686
Z511      488
I110      471
A099      428
I120      420
J440      418
N201      397
I132      396
I678      326
J181      304
E105      272
I200      252
K800      225
K358      220
N40       200
I219      199
N200      178
I771      176
J158      176
E100      173
T814      164
T455      149
I802      148
T824      145
S0680     144
C349      141
L031      138
G448      137
G401      132
D698      126
G408      121
Name: count, dtype: int64

cantidad de clases menores al 10%: 2460
cantidad de clases mayores al 10%: 2494

DX_PRINCIPAL_EGRESO_1
N39    1371
J44    1189
J18    1094
E10     704
N20     668
K80     602
G40     507
Z51     489
I13     486
I11     471
A09     428
I12     425
I67     400
S72     367
I20     366
S06     323
N40     291
S82     278
J15     271
S02     248
S52     241
K35     236
T81     226
I21     215
L03     212
G44     209
I77     204
C34     195
T82     192
K57     190
M86     184
S42     183
K29 

In [ ]:
data_version_1.drop(columns=['DX_PRINCIPAL_EGRESO','DX_Principal_3_Caracteres'],inplace=True)

In [ ]:
#Lista de Letras del codigo DX CIE-10, para la clasificacion de diagnosticos
columnas_diagnosticos = [
    'DX_PRINCIPAL_EGRESO_1', 'DXR_1', 'DXR_2', 'DXR_3',
    'DXR_4', 'DXR_5', 'DXR_6', 'DXR_7', 'DXR_8', 'DXR_9', 'DXR_10'
]
for col in columnas_diagnosticos:
    lista = data_version_1[col].tolist()
    letras = [i[0] for i in lista if isinstance(i, str) and i]  # primera letra si es string no vacío
    unicos = np.unique(letras)
    print(f"{col}: {unicos}")

DX_PRINCIPAL_EGRESO_1: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'Z']
DXR_1: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q'
 'R' 'S' 'T' 'V' 'W' 'X' 'Y' 'Z']
DXR_2: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q'
 'R' 'S' 'T' 'V' 'W' 'X' 'Y' 'Z']
DXR_3: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q'
 'R' 'S' 'T' 'V' 'W' 'X' 'Y' 'Z']
DXR_4: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'Q' 'R'
 'S' 'T' 'V' 'W' 'X' 'Y' 'Z']
DXR_5: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'P' 'Q' 'R'
 'S' 'T' 'V' 'W' 'X' 'Y' 'Z']
DXR_6: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'Q' 'R' 'S'
 'T' 'W' 'X' 'Y' 'Z']
DXR_7: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'Q' 'R' 'S'
 'T' 'W' 'X' 'Y' 'Z']
DXR_8: ['0' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'Q' 'R' 'S'
 'T' 'W' 'X' 'Y' 'Z']
DXR_9: ['0' 'A' 'B' 'C

In [ ]:
#lista de valores mas frecuentes en DX
lista_frecuencias_dx_egreso1 = data_version_1.DX_PRINCIPAL_EGRESO_1.value_counts()[data_version_1.DX_PRINCIPAL_EGRESO_1.value_counts() > 100].index.tolist()
print(lista_frecuencias_dx_egreso1)

['N39', 'J44', 'J18', 'E10', 'N20', 'K80', 'G40', 'Z51', 'I13', 'I11', 'A09', 'I12', 'I67', 'S72', 'I20', 'S06', 'N40', 'S82', 'J15', 'S02', 'S52', 'K35', 'T81', 'I21', 'L03', 'G44', 'I77', 'C34', 'T82', 'K57', 'M86', 'S42', 'K29', 'I80', 'D69', 'T45', 'I48', 'T86', 'J46', 'J21', 'G45', 'B20', 'D38', 'C16', 'C25', 'E11', 'K81', 'K56', 'I25', 'H81', 'M54', 'K83', 'G41', 'K74', 'C18', 'A90', 'K70', 'G43', 'I10', 'I26', 'N13']


In [ ]:
#agrupacion de diagnostico principal dos cifras, la agrupacion reune los grupos medicos por clasificacion hospitalaria, pero dejando por fiera
#los que mas se repiten o de mayor frecuencia por aparte
nueva_llista_dx_egreso = []

for i in data_version_1['DX_PRINCIPAL_EGRESO_1'].tolist():
    if i not in lista_frecuencias_dx_egreso1:
        if not isinstance(i, str) or len(i) < 3:
            nueva_llista_dx_egreso.append('OTROS')
            continue

        letra = i[0].upper()
        try:
            num = int(i[1:3])
        except:
            num = None

        if letra in ['A', 'B']:
            nueva_llista_dx_egreso.append('A-B')
        elif letra == 'C':
            nueva_llista_dx_egreso.append('C-D1')
        elif letra == 'D':
            if num is not None and num <= 48:
                nueva_llista_dx_egreso.append('C-D2')
            else:
                nueva_llista_dx_egreso.append('D-D')
        elif letra == 'E':
            nueva_llista_dx_egreso.append('E-E')
        elif letra == 'F':
            nueva_llista_dx_egreso.append('F-F')
        elif letra == 'G':
            nueva_llista_dx_egreso.append('G-G')
        elif letra == 'H':
            nueva_llista_dx_egreso.append('H-H')
        elif letra == 'I':
            if num is not None and (
                10 <= num <= 15 or
                20 <= num <= 25 or
                60 <= num <= 69
            ):
                nueva_llista_dx_egreso.append('I1')  # Enfermedades cardiovasculares más comunes
            else:
                nueva_llista_dx_egreso.append('I2')  # Otras enfermedades del sistema circulatorio
        elif letra == 'J':
            nueva_llista_dx_egreso.append('J-J')
        elif letra == 'K':
            if num is not None and 0 <= num <= 63:
                nueva_llista_dx_egreso.append('K1')  # Digestivo general
            else:
                nueva_llista_dx_egreso.append('K2')  # Hepatobiliar, etc.
        elif letra == 'L':
            nueva_llista_dx_egreso.append('L-L')
        elif letra == 'M':
            nueva_llista_dx_egreso.append('M-M')
        elif letra == 'N':
            nueva_llista_dx_egreso.append('N-N')
        elif letra in ['O', 'P', 'Q', 'R', 'Z']:
            nueva_llista_dx_egreso.append('P-Q-O-R-Z')
        elif letra == 'S':
            nueva_llista_dx_egreso.append('S-T1')
        elif letra == 'T':
            nueva_llista_dx_egreso.append('S-T2')
        elif letra in ['V', 'W', 'X', 'Y']:
            nueva_llista_dx_egreso.append('S-T3')  # Puedes cambiar este nombre si lo prefieres
        else:
            nueva_llista_dx_egreso.append('OTROS')
    else:
        nueva_llista_dx_egreso.append(i)


data_version_1['DX_PRINCIPAL_EGRESO_1_2'] = nueva_llista_dx_egreso

In [ ]:
print(data_version_1['DX_PRINCIPAL_EGRESO_1_2'].value_counts())

DX_PRINCIPAL_EGRESO_1_2
N39     1371
C-D1    1250
I2      1197
J44     1189
S-T1    1110
        ... 
K70      108
G43      107
I10      106
I26      104
N13      103
Name: count, Length: 80, dtype: int64


In [ ]:
nueva_llista_dx_egreso = []
for i in data_version_1['DX_PRINCIPAL_EGRESO_1'].tolist():
        if not isinstance(i, str) or len(i) < 3:
            nueva_llista_dx_egreso.append('OTROS')
            continue

        letra = i[0].upper()
        try:
            num = int(i[1:3])
        except:
            num = None

        if letra in ['A', 'B']:
            nueva_llista_dx_egreso.append('A-B')
        elif letra == 'C':
            nueva_llista_dx_egreso.append('C-D1')
        elif letra == 'D':
            if num is not None and num <= 48:
                nueva_llista_dx_egreso.append('C-D2')
            else:
                nueva_llista_dx_egreso.append('D-D')
        elif letra == 'E':
            nueva_llista_dx_egreso.append('E-E')
        elif letra == 'F':
            nueva_llista_dx_egreso.append('F-F')
        elif letra == 'G':
            nueva_llista_dx_egreso.append('G-G')
        elif letra == 'H':
            nueva_llista_dx_egreso.append('H-H')
        elif letra == 'I':
            if num is not None and (
                10 <= num <= 15 or
                20 <= num <= 25 or
                60 <= num <= 69
            ):
                nueva_llista_dx_egreso.append('I1')  # Enfermedades cardiovasculares más comunes
            else:
                nueva_llista_dx_egreso.append('I2')  # Otras enfermedades del sistema circulatorio
        elif letra == 'J':
            nueva_llista_dx_egreso.append('J-J')
        elif letra == 'K':
            if num is not None and 0 <= num <= 63:
                nueva_llista_dx_egreso.append('K1')  # Digestivo general
            else:
                nueva_llista_dx_egreso.append('K2')  # Hepatobiliar, etc.
        elif letra == 'L':
            nueva_llista_dx_egreso.append('L-L')
        elif letra == 'M':
            nueva_llista_dx_egreso.append('M-M')
        elif letra == 'N':
            nueva_llista_dx_egreso.append('N-N')
        elif letra in ['O', 'P', 'Q', 'R', 'Z']:
            nueva_llista_dx_egreso.append('P-Q-O-R-Z')
        elif letra == 'S':
            nueva_llista_dx_egreso.append('S-T1')
        elif letra == 'T':
            nueva_llista_dx_egreso.append('S-T2')
        elif letra in ['V', 'W', 'X', 'Y']:
            nueva_llista_dx_egreso.append('S-T3')  # Puedes cambiar este nombre si lo prefieres
        else:
            nueva_llista_dx_egreso.append('OTROS')

data_version_1['DX_PRINCIPAL_EGRESO_1_3'] = nueva_llista_dx_egreso

In [ ]:
print(data_version_1['DX_PRINCIPAL_EGRESO_1_3'].value_counts())

DX_PRINCIPAL_EGRESO_1_3
J-J          3666
N-N          3129
I1           2990
S-T1         2750
I2           1813
C-D1         1799
K2           1621
K1           1570
G-G          1465
A-B          1274
S-T2         1222
E-E          1046
M-M           987
P-Q-O-R-Z     663
C-D2          564
L-L           522
D-D           372
H-H           279
F-F           137
Name: count, dtype: int64


In [ ]:
reagrupacion = {
    'J-J': 'Respiratorio',
    'N-N': 'Genitourinario',
    'I1': 'Circulatorio',
    'I2': 'Circulatorio',
    'S-T1': 'Traumatismo',
    'S-T2': 'Traumatismo',
    'C-D1': 'Neoplasias',
    'C-D2': 'Neoplasias',
    'D-D': 'Neoplasias',
    'A-B': 'Infecciosas',
    'K1': 'Digestivo',
    'K2': 'Digestivo',
    'G-G': 'Neurológico',
    'F-F': 'Psicológico',
    'E-E': 'Endocrino',
    'M-M': 'Musculoesquelético',
    'L-L': 'Dermatológico',
    'H-H': 'Sensorial',
    'P-Q-O-R-Z': 'Otros'
}
data_version_1['DX_EGRESO_AGRUPADO'] = data_version_1['DX_PRINCIPAL_EGRESO_1_3'].map(reagrupacion)

In [ ]:
print(data_version_1['DX_EGRESO_AGRUPADO'].value_counts())

DX_EGRESO_AGRUPADO
Circulatorio          4803
Traumatismo           3972
Respiratorio          3666
Digestivo             3191
Genitourinario        3129
Neoplasias            2735
Neurológico           1465
Infecciosas           1274
Endocrino             1046
Musculoesquelético     987
Otros                  663
Dermatológico          522
Sensorial              279
Psicológico            137
Name: count, dtype: int64


In [ ]:
#comparacion entre el de dos cifras sin agrupar y agrupado de las dos formas
chi2_y_cramers_v(data_version_1,'DX_PRINCIPAL_EGRESO_1', 'GRD_CODIGO')
chi2_y_cramers_v(data_version_1,'DX_PRINCIPAL_EGRESO_1_2', 'GRD_CODIGO')
chi2_y_cramers_v(data_version_1,'DX_PRINCIPAL_EGRESO_1_3', 'GRD_CODIGO')
chi2_y_cramers_v(data_version_1,'DX_EGRESO_AGRUPADO', 'GRD_CODIGO')

DX_PRINCIPAL_EGRESO_1
Estadístico chi² = 3765620.94
p-valor = 0.0000
V de Cramer = 0.4718
DX_PRINCIPAL_EGRESO_1_2
Estadístico chi² = 1187618.65
p-valor = 0.0000
V de Cramer = 0.7345
DX_PRINCIPAL_EGRESO_1_3
Estadístico chi² = 364478.98
p-valor = 0.0000
V de Cramer = 0.8524
DX_EGRESO_AGRUPADO
Estadístico chi² = 275195.58
p-valor = 0.0000
V de Cramer = 0.8715


In [ ]:
data_version_1.drop(columns=['DX_PRINCIPAL_EGRESO_1','DX_PRINCIPAL_EGRESO_1_2','DX_PRINCIPAL_EGRESO_1_3'],inplace=True)

**Análisis:**
La última agrupación evaluada resultó ser la más adecuada, ya que presentó los valores más altos tanto en la prueba de Chi-cuadrado como en el coeficiente de Cramér. Esto indica una mejor asociación con la variable objetivo, por lo que se seleccionará esta agrupación para representar los diagnósticos en el modelo.

In [ ]:
#Visualizacion estadistica de los datos antes de aplicar agrupacion para reducir el desbalance
for i in range(1,11):
  i_='DXR_'+str(i)
  chi2_y_cramers_v(data_version_1,i_, 'GRD_CODIGO')

DXR_1
Estadístico chi² = 3002022.84
p-valor = 0.0000
V de Cramer = 0.4213
DXR_2
Estadístico chi² = 1867785.64
p-valor = 0.0000
V de Cramer = 0.3323
DXR_3
Estadístico chi² = 1527709.82
p-valor = 0.0000
V de Cramer = 0.3005
DXR_4
Estadístico chi² = 1223620.67
p-valor = 0.0000
V de Cramer = 0.2689
DXR_5
Estadístico chi² = 929795.49
p-valor = 0.0000
V de Cramer = 0.2344
DXR_6
Estadístico chi² = 752721.17
p-valor = 0.0000
V de Cramer = 0.2109
DXR_7
Estadístico chi² = 606757.94
p-valor = 0.0000
V de Cramer = 0.1894
DXR_8
Estadístico chi² = 476599.81
p-valor = 0.0000
V de Cramer = 0.1679
DXR_9
Estadístico chi² = 347339.71
p-valor = 0.0000
V de Cramer = 0.1568
DXR_10
Estadístico chi² = 524.85
p-valor = 0.9929
V de Cramer = 0.1372


In [ ]:
#Visualizacion estadistica de los datos después de aplicar agrupacion para reducir el desbalance
for i in range(1,11):
  i_='DXR_'+str(i)
  agrupar_dx(data_version_1,i_)
  chi2_y_cramers_v(data_version_1,i_, 'GRD_CODIGO')


DXR_1
Estadístico chi² = 96530.31
p-valor = 0.0000
V de Cramer = 0.4974
DXR_2
Estadístico chi² = 47561.69
p-valor = 0.0000
V de Cramer = 0.3491
DXR_3
Estadístico chi² = 32663.38
p-valor = 0.0000
V de Cramer = 0.2893
DXR_4
Estadístico chi² = 25326.67
p-valor = 0.0000
V de Cramer = 0.2548
DXR_5
Estadístico chi² = 22091.60
p-valor = 0.0000
V de Cramer = 0.2380
DXR_6
Estadístico chi² = 18956.16
p-valor = 0.0000
V de Cramer = 0.2204
DXR_7
Estadístico chi² = 16123.69
p-valor = 0.0000
V de Cramer = 0.2033
DXR_8
Estadístico chi² = 14636.05
p-valor = 0.0000
V de Cramer = 0.1937
DXR_9
Estadístico chi² = 12944.19
p-valor = 0.0000
V de Cramer = 0.1821
DXR_10
Estadístico chi² = 524.85
p-valor = 0.9929
V de Cramer = 0.1372


In [ ]:
data_version_1.drop(columns=['DXR_10'],inplace=True)

**Analisis:**
De manera similar, se aplicó la misma metodología de agrupación a las comorbilidades (DXR), que utilizan la misma codificación. Se observó una mejora en la relación con la variable objetivo, aunque no tan significativa como en el diagnóstico principal. La única comorbilidad que no aporta valor predictivo es la número 10, lo cual es consistente dado que solo cuenta con un caso único en el conjunto de datos.

###**Analisis de Procedimientos en Estancia CIE-9_MC**

Los procedimientos realizados durante la estancia del paciente están codificados según la norma CIE-9-MC, la cual establece un sistema estandarizado para clasificar intervenciones quirúrgicas y terapéuticas. Estos códigos abarcan un rango que va desde 00.00 hasta 99.99, permitiendo representar de forma detallada una amplia variedad de procedimientos clínicos.

Dado el amplio espectro de códigos posibles, fue necesario verificar el valor máximo y el rango de códigos presentes en cada una de las columnas de procedimientos. Sin embargo, al igual que los diagnósticos, estos códigos presentan una alta cardinalidad, lo que representa un desafío para su incorporación directa en modelos predictivos. Por esta razón, se hace indispensable diseñar una estrategia de agrupación adecuada, que permita reducir la dimensionalidad sin perder información clínica relevante.

In [ ]:
for i in range(1,31):
  i_='PROC_'+str(i)
  proc_lista = data_version_1[i_].tolist()
  proc_list_numerica = [float(x) if x != 'VACIO' else np.nan for x in proc_lista]
  print(i_)
  print(np.nanmax(proc_list_numerica))
  print(np.nanmin(proc_list_numerica))
  print('')

PROC_1
99.62
0.0

PROC_2
98.2
0.66

PROC_3
97.82
0.66

PROC_4
97.23
0.66

PROC_5
96.72
0.0

PROC_6
96.72
0.66

PROC_7
96.72
0.66

PROC_8
96.72
0.66

PROC_9
96.72
0.68

PROC_10
96.72
0.68

PROC_11
97.23
0.68

PROC_12
96.72
0.68

PROC_13
96.72
0.68

PROC_14
96.72
0.68

PROC_15
96.72
0.68

PROC_16
96.72
0.68

PROC_17
96.72
0.68

PROC_18
96.72
0.68

PROC_19
96.72
0.68

PROC_20
96.72
11.51

PROC_21
96.71
14.71

PROC_22
96.71
0.68

PROC_23
96.72
86.22

PROC_24
96.72
0.68

PROC_25
93.08
38.03

PROC_26
89.51
38.05

PROC_27
93.08
87.03

PROC_28
96.72
87.03

PROC_29
88.77
88.01

PROC_30
89.58
88.21



In [ ]:
#Visualizacion estadistica de los datos antes de aplicar agrupacion para reducir el desbalance
for i in range(1,31):
  i_='PROC_'+str(i)
  chi2_y_cramers_v(data_version_1,i_, 'GRD_CODIGO')
  print('')

PROC_1
Estadístico chi² = 2027478.34
p-valor = 0.0000
V de Cramer = 0.3462

PROC_2
Estadístico chi² = 1219886.33
p-valor = 0.0000
V de Cramer = 0.2685

PROC_3
Estadístico chi² = 730519.47
p-valor = 0.0000
V de Cramer = 0.2256

PROC_4
Estadístico chi² = 490156.57
p-valor = 0.0000
V de Cramer = 0.2027

PROC_5
Estadístico chi² = 339094.44
p-valor = 0.0000
V de Cramer = 0.1950

PROC_6
Estadístico chi² = 259943.19
p-valor = 0.0000
V de Cramer = 0.1832

PROC_7
Estadístico chi² = 216639.65
p-valor = 0.0000
V de Cramer = 0.1875

PROC_8
Estadístico chi² = 198858.86
p-valor = 0.0000
V de Cramer = 0.1866

PROC_9
Estadístico chi² = 155761.14
p-valor = 0.0000
V de Cramer = 0.1782

PROC_10
Estadístico chi² = 131985.21
p-valor = 0.0000
V de Cramer = 0.1731

PROC_11
Estadístico chi² = 118203.35
p-valor = 0.0000
V de Cramer = 0.1779

PROC_12
Estadístico chi² = 88697.84
p-valor = 0.0000
V de Cramer = 0.1749

PROC_13
Estadístico chi² = 67707.56
p-valor = 0.0000
V de Cramer = 0.1643

PROC_14
Estadístico c

In [ ]:
#Visualizacion estadistica de los datos después de aplicar agrupacion para reducir el desbalance
for i in range(1,31):
  i_='PROC_'+str(i)
  data_version_1[i_+'_GRUPO'] = data_version_1[i_].apply(agrupar_cie9_mc)
  chi2_y_cramers_v(data_version_1,i_+'_GRUPO', 'GRD_CODIGO')
  print('')

PROC_1_GRUPO
Estadístico chi² = 202076.96
p-valor = 0.0000
V de Cramer = 0.6732

PROC_2_GRUPO
Estadístico chi² = 107460.63
p-valor = 0.0000
V de Cramer = 0.5070

PROC_3_GRUPO
Estadístico chi² = 60942.19
p-valor = 0.0000
V de Cramer = 0.3818

PROC_4_GRUPO
Estadístico chi² = 46235.97
p-valor = 0.0000
V de Cramer = 0.3326

PROC_5_GRUPO
Estadístico chi² = 36945.60
p-valor = 0.0000
V de Cramer = 0.3077

PROC_6_GRUPO
Estadístico chi² = 23548.06
p-valor = 0.0000
V de Cramer = 0.2549

PROC_7_GRUPO
Estadístico chi² = 23456.48
p-valor = 0.0000
V de Cramer = 0.2452

PROC_8_GRUPO
Estadístico chi² = 23595.48
p-valor = 0.0000
V de Cramer = 0.2459

PROC_9_GRUPO
Estadístico chi² = 18324.83
p-valor = 0.0000
V de Cramer = 0.2249

PROC_10_GRUPO
Estadístico chi² = 14836.30
p-valor = 0.0000
V de Cramer = 0.2024

PROC_11_GRUPO
Estadístico chi² = 15520.56
p-valor = 0.0000
V de Cramer = 0.2070

PROC_12_GRUPO
Estadístico chi² = 13467.14
p-valor = 0.0000
V de Cramer = 0.1928

PROC_13_GRUPO
Estadístico chi² = 11

In [ ]:
data_version_1.drop(columns=['PROC_1','PROC_2','PROC_3','PROC_4','PROC_5','PROC_6','PROC_7','PROC_8','PROC_9','PROC_10','PROC_11','PROC_12',
                             'PROC_13','PROC_14','PROC_15','PROC_16','PROC_17','PROC_18','PROC_19','PROC_20','PROC_21','PROC_22','PROC_23',
                             'PROC_24','PROC_25','PROC_26','PROC_27','PROC_28','PROC_29','PROC_30','PROC_24_GRUPO',
                             'PROC_25_GRUPO','PROC_26_GRUPO','PROC_27_GRUPO','PROC_28_GRUPO','PROC_29_GRUPO','PROC_30_GRUPO'],inplace=True)

In [ ]:
data_version_1.PROC_1_GRUPO.value_counts()

,count
PROC_1_GRUPO,
Desconocido,8707
Diagnóstico/terapéutico,8236
Genitourinario,4390
Cirugía músculo-esquelética,2308
Digestivo,1368
Parto y puerperio,681
Linfático y bazo,605
Sistema nervioso,325
Mama y piel,323


**Analisis:**

Gracias a la agrupación de clases, se logró reducir la alta cardinalidad presente en los códigos de procedimientos, lo que a su vez mejoró su relación con la variable objetivo. No obstante, las columnas correspondientes a los procedimientos 24 al 30 continuaron mostrando una escasa relevancia para la predicción, por lo que se optó por su eliminación del modelo.

###**Analisis Servicios Hospital**

In [ ]:
print(data_version_1['TIPO_INGRESO'].value_counts())
print('')
print(data_version_1['TIPO_SERVICIO'].value_counts())
print('')
print(data_version_1['CUIDADOS_INTENSIVOS'].value_counts())
print('')
print(data_version_1['SERVICIO_ALTA'].value_counts())
print('')
print(data_version_1['SITUACION_ALTA'].value_counts())

TIPO_INGRESO
URGENCIA      25570
PROGRAMADO     2294
urgencias         3
Urgencias         1
Programado        1
Name: count, dtype: int64

TIPO_SERVICIO
URGENCIA ADULTOS        24836
NO APLICA                2348
URGENCIA PEDÍATRICAS      367
CIRUGÍA                   308
U. ADULTOS                  1
URGENCIA ADULTO             1
URGENCIA ADULTOS.           1
URGENCIA - ADULTOS          1
No aplica                   1
N0 APLICA                   1
CIRUGIA                     1
URGENCIA PEDIATRICAS        1
ATENCIÓN PRIORITARIA        1
Name: count, dtype: int64

CUIDADOS_INTENSIVOS
NO    26681
SI     1182
no        3
Si        1
No        1
si        1
Name: count, dtype: int64

SERVICIO_ALTA
156    2031
24     1856
162    1699
20     1685
23     1617
126    1571
55     1569
59     1521
28     1504
44     1492
22     1372
297    1351
54     1233
38     1226
39     1194
29     1134
40      970
21      899
31      758
30      509
158     326
25      123
42       98
58       88
45      

In [ ]:
#Correccion de clases:
data_version_1.TIPO_INGRESO=data_version_1.TIPO_INGRESO.replace({'urgencias':'URGENCIA','Urgencias':'URGENCIA','Programado':'PROGRAMADO'})
data_version_1.TIPO_SERVICIO=data_version_1.TIPO_SERVICIO.replace({'U. ADULTOS':'URGENCIA ADULTOS','URGENCIA ADULTO':'URGENCIA ADULTOS','URGENCIA ADULTOS.':'URGENCIA ADULTOS','URGENCIA - ADULTOS':'URGENCIA ADULTOS','N0 APLICA':'NO APLICA','No aplica': 'NO APLICA','CIRUGIA':'CIRUGÍA','URGENCIA PEDIATRICAS':'URGENCIA PEDÍATRICAS','ATENCIÓN PRIORITARIA':'NO APLICA'})
data_version_1.CUIDADOS_INTENSIVOS=data_version_1.CUIDADOS_INTENSIVOS.replace({'si':'SI','Si':'SI','no':'NO','No':'NO'})
data_version_1.SITUACION_ALTA=data_version_1.SITUACION_ALTA.replace({'ALTA MEDICA':'ALTA MÉDICA','Muerto':'FALLECIDO','Fallecido':'FALLECIDO','fallecido':'FALLECIDO'})
print(data_version_1['TIPO_INGRESO'].value_counts())
print('')
print(data_version_1['TIPO_SERVICIO'].value_counts())
print('')
print(data_version_1['CUIDADOS_INTENSIVOS'].value_counts())
print('')
print(data_version_1['SITUACION_ALTA'].value_counts())



TIPO_INGRESO
URGENCIA      25574
PROGRAMADO     2295
Name: count, dtype: int64

TIPO_SERVICIO
URGENCIA ADULTOS        24840
NO APLICA                2351
URGENCIA PEDÍATRICAS      368
CIRUGÍA                   309
Name: count, dtype: int64

CUIDADOS_INTENSIVOS
NO    26685
SI     1184
Name: count, dtype: int64

SITUACION_ALTA
ALTA MÉDICA    26373
FALLECIDO       1495
Name: count, dtype: int64


In [ ]:
#Llenar los nulos  de las categorias alta medica y tipo de servicio
data_version_1.SITUACION_ALTA=data_version_1.SITUACION_ALTA.fillna('ALTA MÉDICA')
data_version_1.TIPO_SERVICIO=data_version_1.TIPO_SERVICIO.fillna('NO APLICA')

In [ ]:
chi2_y_cramers_v(data_version_1,'TIPO_INGRESO', 'GRD_CODIGO')
print('')
chi2_y_cramers_v(data_version_1,'TIPO_SERVICIO', 'GRD_CODIGO')
print('')
chi2_y_cramers_v(data_version_1,'CUIDADOS_INTENSIVOS', 'GRD_CODIGO')
print('')
chi2_y_cramers_v(data_version_1,'SITUACION_ALTA', 'GRD_CODIGO')
print('')
chi2_y_cramers_v(data_version_1,'SERVICIO_ALTA', 'GRD_CODIGO')

TIPO_INGRESO
Estadístico chi² = 10381.03
p-valor = 0.0000
V de Cramer = 0.6103

TIPO_SERVICIO
Estadístico chi² = 10924.20
p-valor = 0.0000
V de Cramer = 0.3615

CUIDADOS_INTENSIVOS
Estadístico chi² = 9766.48
p-valor = 0.0000
V de Cramer = 0.5920

SITUACION_ALTA
Estadístico chi² = 3687.46
p-valor = 0.0000
V de Cramer = 0.3637

SERVICIO_ALTA
Estadístico chi² = 75460.72
p-valor = 0.0000
V de Cramer = 0.3291


In [ ]:
# Agrupar servicios con menos de 200 casos como "OTROS"
frecuencias = data_version_1['SERVICIO_ALTA'].value_counts()
servicios_comunes = frecuencias[frecuencias > 400].index
data_version_1['SERVICIO_ALTA_AGRUPADO'] = data_version_1['SERVICIO_ALTA'].apply(
    lambda x: x if x in servicios_comunes else 'OTROS'
)


In [ ]:
chi2_y_cramers_v(data_version_1,'SERVICIO_ALTA_AGRUPADO', 'GRD_CODIGO')

SERVICIO_ALTA_AGRUPADO
Estadístico chi² = 69903.40
p-valor = 0.0000
V de Cramer = 0.3541


###**Infecciones**

In [ ]:
#Visualizacion de las columnas relacionadas con infecciones
print(data_version_1.INFECCIONES.value_counts())
print('')
print(data_version_1.INFECCION_QUIRURGICA.value_counts())

INFECCIONES
NO    27356
SI      513
Name: count, dtype: int64

INFECCION_QUIRURGICA
NO    27663
SI      206
Name: count, dtype: int64


In [ ]:
#Se reescribe las clases mal escritas de las columnas
data_version_1['INFECCIONES'] = data_version_1['INFECCIONES'].replace({'si':'SI', 'no':'NO'})
data_version_1['INFECCION_QUIRURGICA'] = data_version_1['INFECCION_QUIRURGICA'].replace({'Si':'SI', 'si':'SI'})
print(data_version_1.INFECCIONES.value_counts())
print('')
print(data_version_1.INFECCION_QUIRURGICA.value_counts())

INFECCIONES
NO    27356
SI      513
Name: count, dtype: int64

INFECCION_QUIRURGICA
NO    27663
SI      206
Name: count, dtype: int64


In [ ]:
#Comprobacion de la relacion entre las dos columnas de infecciones
#Ambas columnas estan relacionadas siendo una, una rama de la otra, por lo que se puede quedar con una
infecciones= data_version_1.apply(
    lambda row: 'SI' if row['INFECCIONES'] == 'SI' or row['INFECCION_QUIRURGICA'] == 'SI' else 'NO',
    axis=1
)
print(infecciones.value_counts())

NO    27356
SI      513
Name: count, dtype: int64


In [ ]:
chi2_y_cramers_v(data_version_1,'INFECCIONES', 'GRD_CODIGO')
print('')
chi2_y_cramers_v(data_version_1,'INFECCION_QUIRURGICA', 'GRD_CODIGO')

INFECCIONES
Estadístico chi² = 2718.82
p-valor = 0.0000
V de Cramer = 0.3123

INFECCION_QUIRURGICA
Estadístico chi² = 3756.05
p-valor = 0.0000
V de Cramer = 0.3671


**Analisis:**Las dos columnas de infecciones son dos caracteristicas de clases binarias, las cuales informan si hubo una infección en la estadia en el hospital, una informa de forma general las infecciones en la estadia(INFECCIONES), la otra es un subconjunto la cual informa las infecciones ocurridas al momento de una cirugia(INFECCIONES_QUIRURGICA)

###**Edad y Sexo**

In [ ]:
print(data_version_1.EDAD.value_counts())
print('')
chi2_y_cramers_v(data_version_1,'EDAD', 'GRD_CODIGO')

EDAD
81     592
77     586
68     555
76     551
75     540
      ... 
102      2
106      2
105      1
109      1
104      1
Name: count, Length: 108, dtype: int64

EDAD
Estadístico chi² = 85402.74
p-valor = 0.0000
V de Cramer = 0.1692


In [ ]:
def agrupar_edad(df, nombre_variable_edad, metodo='decadas'):
    df = df.copy()

    if metodo == 'decadas':
        bins = list(range(0, 101, 10)) + [150]
        labels = [f"{i}-{i+9}" for i in range(0, 100, 10)] + ['100+']  # 11 labels
        df['EDAD_GRUPO'] = pd.cut(df[nombre_variable_edad], bins=bins, right=False, labels=labels)

    elif metodo == 'clinico':
        bins = [-1, 1, 12, 17, 39, 64, 150]
        labels = ['Neonato', 'Niñez', 'Adolescencia', 'Joven adulto', 'Adulto', 'Adulto mayor']  # 6 labels
        df['EDAD_GRUPO'] = pd.cut(df[nombre_variable_edad], bins=bins, labels=labels)

    elif metodo == 'cuartiles':
        df['EDAD_GRUPO'] = pd.qcut(df[nombre_variable_edad], q=4, duplicates='drop')

    elif metodo == 'quintiles':
        df['EDAD_GRUPO'] = pd.qcut(df[nombre_variable_edad], q=5, duplicates='drop')

    elif metodo == 'etapas_vida':
        bins = [0, 1, 5, 12, 18, 30, 45, 60, 75, 90, 120]
        labels = ['Lactante', 'Preescolar', 'Escolar', 'Adolescente', 'Joven adulto',
                  'Adulto', 'Mediana edad', 'Adulto mayor', 'Anciano', 'Muy anciano']  # 10 labels
        df['EDAD_GRUPO'] = pd.cut(df[nombre_variable_edad], bins=bins, labels=labels)

    elif metodo == 'mixto':
        bins = [0, 1, 10, 20, 40, 60, 75, 85, 120]
        labels = ['Neonatal', 'Infantil', 'Niño', 'Joven', 'Adulto joven',
                  'Adulto medio', 'Adulto mayor', 'Anciano']  # 8 labels
        df['EDAD_GRUPO'] = pd.cut(df[nombre_variable_edad], bins=bins, labels=labels)

    else:
        raise ValueError("Método de agrupación no reconocido.")

    # Calcula Chi2 y Cramér's V
    chi2_y_cramers_v(df, 'EDAD_GRUPO', 'GRD_CODIGO')

    return df



In [ ]:
data_version_1_decades = agrupar_edad(data_version_1, 'EDAD', metodo='decadas')
print('')
data_version_1_clinico = agrupar_edad(data_version_1, 'EDAD',  metodo='clinico')
print('')
data_version_1_cuartiles = agrupar_edad(data_version_1, 'EDAD',  metodo='cuartiles')
print('')
data_version_1_quintiles = agrupar_edad(data_version_1, 'EDAD',  metodo='quintiles')
print('')
data_version_1_etapas_vida = agrupar_edad(data_version_1, 'EDAD',  metodo='etapas_vida')
print('')
data_version_1_mixto = agrupar_edad(data_version_1, 'EDAD',  metodo='mixto')

EDAD_GRUPO
Estadístico chi² = 24567.44
p-valor = 0.0000
V de Cramer = 0.2969

EDAD_GRUPO
Estadístico chi² = 20817.21
p-valor = 0.0000
V de Cramer = 0.3865

EDAD_GRUPO
Estadístico chi² = 11876.41
p-valor = 0.0000
V de Cramer = 0.3769

EDAD_GRUPO
Estadístico chi² = 13536.73
p-valor = 0.0000
V de Cramer = 0.3485

EDAD_GRUPO
Estadístico chi² = 24690.98
p-valor = 0.0000
V de Cramer = 0.3140

EDAD_GRUPO
Estadístico chi² = 22693.75
p-valor = 0.0000
V de Cramer = 0.3413


In [ ]:
print(data_version_1.SEXO.value_counts())
print('')
chi2_y_cramers_v(data_version_1,'SEXO', 'GRD_CODIGO')

SEXO
M    14353
F    13512
f        2
m        2
Name: count, dtype: int64

SEXO
Estadístico chi² = 3121.84
p-valor = 0.0000
V de Cramer = 0.1932


In [ ]:
data_version_1=data_version_1.replace({'f':'F','m':'M'})
print(data_version_1.SEXO.value_counts())
print('')
chi2_y_cramers_v(data_version_1,'SEXO', 'GRD_CODIGO')

SEXO
M    14355
F    13514
Name: count, dtype: int64

SEXO
Estadístico chi² = 2417.19
p-valor = 0.0000
V de Cramer = 0.2945


**Analisis:**

Se realizaron ajustes en dos columnas: sexo y edad. En la columna de sexo, se corrigieron valores mal escritos para asegurar consistencia en las categorías. En cuanto a la columna de edad, se redefinieron los rangos etarios con el objetivo de reducir el desbalance entre grupos. Esta nueva agrupación fue determinada cuidadosamente y justificada mediante pruebas estadísticas, incluyendo el coeficiente de asociación de Cramér, garantizando una representación más equitativa sin comprometer la integridad analítica de los datos.

###**Estancia**

In [ ]:
print(data_version_1.DIAS_ESTANCIA.value_counts())
print('')
chi2_y_cramers_v(data_version_1,'DIAS_ESTANCIA', 'GRD_CODIGO')

DIAS_ESTANCIA
3      3161
2      3076
4      2916
5      2640
6      2085
       ... 
83        1
106       1
96        1
174       1
127       1
Name: count, Length: 107, dtype: int64

DIAS_ESTANCIA
Estadístico chi² = 96670.18
p-valor = 0.0000
V de Cramer = 0.1809


In [ ]:
def agrupar_estancia(df, nombre_variable_estancia, metodo='clinico'):
    df = df.copy()

    if metodo == 'clinico':
        bins = [-1, 1, 4, 7, 14, 999]
        labels = ['<2 días', '2-4 días', '5-7 días', '8-14 días', '15+ días']
        df['ESTANCIA_GRUPO'] = pd.cut(df[nombre_variable_estancia], bins=bins, labels=labels)

    elif metodo == 'cuartiles':
        df['ESTANCIA_GRUPO'] = pd.qcut(df[nombre_variable_estancia], q=4, duplicates='drop')

    elif metodo == 'quintiles':
        df['ESTANCIA_GRUPO'] = pd.qcut(df[nombre_variable_estancia], q=5, duplicates='drop')

    elif metodo == 'log':
        import numpy as np
        df['ESTANCIA_GRUPO'] = pd.cut(np.log1p(df[nombre_variable_estancia]),
                                      bins=[0, 1, 2, 3, 4, 10],
                                      labels=['muy corta', 'corta', 'media', 'larga', 'muy larga'])

    else:
        raise ValueError("Método de agrupación no reconocido.")

    # Evaluación
    chi2_y_cramers_v(df, 'ESTANCIA_GRUPO', 'GRD_CODIGO')

    return df


In [ ]:
data_version_1_clinico = agrupar_estancia(data_version_1, 'DIAS_ESTANCIA',  metodo='clinico')
print('')
data_version_1_cuartiles = agrupar_estancia(data_version_1, 'DIAS_ESTANCIA',  metodo='cuartiles')
print('')
data_version_1_quintiles = agrupar_estancia(data_version_1, 'DIAS_ESTANCIA',  metodo='quintiles')
print('')
data_version_1_etapas_vida = agrupar_estancia(data_version_1, 'DIAS_ESTANCIA',  metodo='log')

ESTANCIA_GRUPO
Estadístico chi² = 14972.94
p-valor = 0.0000
V de Cramer = 0.3665

ESTANCIA_GRUPO
Estadístico chi² = 11621.45
p-valor = 0.0000
V de Cramer = 0.3728

ESTANCIA_GRUPO
Estadístico chi² = 12920.29
p-valor = 0.0000
V de Cramer = 0.3404

ESTANCIA_GRUPO
Estadístico chi² = 15420.92
p-valor = 0.0000
V de Cramer = 0.3720


In [ ]:
print(data_version_1.DIAS_CUIDADOS_INTENSIVOS.value_counts())
print('')
chi2_y_cramers_v(data_version_1,'DIAS_CUIDADOS_INTENSIVOS', 'GRD_CODIGO')

DIAS_CUIDADOS_INTENSIVOS
0     26685
2       242
3       140
4       108
1        98
      ...  
34        1
63        1
51        1
43        1
72        1
Name: count, Length: 66, dtype: int64

DIAS_CUIDADOS_INTENSIVOS
Estadístico chi² = 64231.31
p-valor = 0.0000
V de Cramer = 0.1883


In [ ]:
def agrupar_cuidados_intensivos(df, nombre_variable='DIAS_CUIDADOS_INTENSIVOS', metodo='clinico'):
    df = df.copy()

    if metodo == 'clinico':
        bins = [-1, 0, 1, 4, 9, 999]
        labels = ['Sin cuidados', '1 día', '2-4 días', '5-9 días', '10+ días']
        df['UCI_GRUPO'] = pd.cut(df[nombre_variable], bins=bins, labels=labels)

    elif metodo == 'cuartiles':
        df['UCI_GRUPO'] = pd.qcut(df[nombre_variable], q=4, duplicates='drop')

    elif metodo == 'log':
        import numpy as np
        df['UCI_GRUPO'] = pd.cut(np.log1p(df[nombre_variable]),
                                 bins=[0, 0.5, 1.5, 2.5, 4, 10],
                                 labels=['0', '1', '2-3', '4-10', '10+'])

    else:
        raise ValueError("Método no reconocido.")

    # Evaluación de relación con GRD
    chi2_y_cramers_v(df, 'UCI_GRUPO', 'GRD_CODIGO')

    return df


In [ ]:
data_version_1_clinico2 = agrupar_cuidados_intensivos(data_version_1, 'DIAS_CUIDADOS_INTENSIVOS',  metodo='clinico')
print('')
data_version_1_cuartiles2 = agrupar_cuidados_intensivos(data_version_1, 'DIAS_CUIDADOS_INTENSIVOS',  metodo='cuartiles')
print('')
data_version_1_etapas_vida2 = agrupar_cuidados_intensivos(data_version_1, 'DIAS_CUIDADOS_INTENSIVOS',  metodo='log')

UCI_GRUPO
Estadístico chi² = 20461.96
p-valor = 0.0000
V de Cramer = 0.4284

UCI_GRUPO
Estadístico chi² = 0.00
p-valor = 1.0000
V de Cramer = nan

UCI_GRUPO
Estadístico chi² = 1001.03
p-valor = 0.0000
V de Cramer = 0.5309


/tmp/ipython-input-7-1118822241.py:8: RuntimeWarning: invalid value encountered in divide
  v_cramer = np.sqrt(chi2 / (n * (min(r, k) - 1)))


**Analisis**

Para las variables días de estancia y días en UCI, se realizó una nueva agrupación de los valores con el objetivo de mejorar la distribución y representatividad de los datos. Cada agrupación propuesta fue evaluada mediante la prueba de chi-cuadrado y el coeficiente de Cramér, seleccionando finalmente aquella que mostró una mejor asociación con la variable dependiente y redujo el desbalance entre categorías.

###**Fecha**

In [ ]:
# Asegurarnos que la columna sea string
data_version_1["FECHA_INGRESO"] = data_version_1["FECHA_INGRESO"].astype(str)

# Rellenar con ceros a la izquierda para que tenga 8 caracteres (ddmmaaaa)
data_version_1["FECHA_INGRESO"] = data_version_1["FECHA_INGRESO"].apply(lambda x: x.zfill(8))

# Crear nuevas columnas separadas para día, mes y año
data_version_1["DIA"] = data_version_1["FECHA_INGRESO"].str[:2]
data_version_1["MES"] = data_version_1["FECHA_INGRESO"].str[2:4]
data_version_1["AÑO"] = data_version_1["FECHA_INGRESO"].str[4:]

# Eliminar la columna original si ya no la necesitas
data_version_1.pop("FECHA_INGRESO")

# Combinar las columnas para crear una fecha tipo datetime
data_version_1["FECHA_DATETIME"] = pd.to_datetime(
    data_version_1["DIA"] + data_version_1["MES"] + data_version_1["AÑO"],
    format="%d%m%Y",
    errors='coerce'
)

# Extraer el día de la semana en inglés
data_version_1["DIA_SEMANA"] = data_version_1["FECHA_DATETIME"].dt.day_name()

# Traducir el día de la semana al español
dias_es = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miércoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}
data_version_1["DIA_SEMANA"] = data_version_1["DIA_SEMANA"].map(dias_es)
data_version_1.drop(columns=["DIA", "FECHA_DATETIME", "AÑO"], inplace=True)

##**Features**

In [ ]:
data_version_1['EDAD_AGRUPADA'] = agrupar_edad(data_version_1, 'EDAD',  metodo='clinico')['EDAD_GRUPO']

EDAD_GRUPO
Estadístico chi² = 20817.21
p-valor = 0.0000
V de Cramer = 0.3865


In [ ]:
data_version_1['ESTANCIA_AGRUPADA'] = agrupar_estancia(data_version_1, 'DIAS_ESTANCIA',  metodo='clinico')['ESTANCIA_GRUPO']
data_version_1['UCI_AGRUPADA'] = agrupar_cuidados_intensivos(data_version_1, 'DIAS_CUIDADOS_INTENSIVOS',  metodo='clinico')['UCI_GRUPO']

ESTANCIA_GRUPO
Estadístico chi² = 14972.94
p-valor = 0.0000
V de Cramer = 0.3665
UCI_GRUPO
Estadístico chi² = 20461.96
p-valor = 0.0000
V de Cramer = 0.4284


In [ ]:
#Eliminar COlumnas
data_version_1.drop(columns=['SERVICIO_ALTA','DX_INGRESO','FECHA_EGRESO','EDAD','DIAS_CUIDADOS_INTENSIVOS','DIAS_ESTANCIA'],inplace=True)

In [ ]:
resumen(data_version_1)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,object,2,27869,0,0.00%
TIPO_INGRESO,object,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,object,2,27869,0,0.00%
DXR_1,object,15,27869,0,0.00%
DXR_2,object,15,27869,0,0.00%
DXR_3,object,15,27869,0,0.00%
DXR_4,object,15,27869,0,0.00%
DXR_5,object,15,27869,0,0.00%
DXR_6,object,15,27869,0,0.00%
DXR_7,object,15,27869,0,0.00%



🧾 El DataFrame tiene un total de **49** columnas y **27869** filas.


In [ ]:
X=data_version_1.drop(columns=['GRD_CODIGO','TIPO_GRD','GRD_DESCRIPCION'])
y=data_version_1['GRD_CODIGO']

In [ ]:
def cramers_v_corrected(confusion_matrix):
    chi2, _, _, _ = chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

def encontrar_columnas_redundantes(df, umbral=0.60):
    cols = df.columns
    redundantes = []
    revisadas = set()

    for i, col1 in enumerate(cols):
        for j in range(i + 1, len(cols)):
            col2 = cols[j]
            pair = (col1, col2)
            if pair not in revisadas:
                try:
                    tabla = pd.crosstab(df[col1], df[col2])
                    if tabla.shape[0] > 1 and tabla.shape[1] > 1:
                        v = cramers_v_corrected(tabla)
                        if v > umbral:
                            redundantes.append((col1, col2, round(v, 3)))
                except Exception as e:
                    pass
                revisadas.add(pair)
    return redundantes


In [ ]:
def encontrar_columnas_numericas_redundantes(df_numericas, umbral=0.85):
    correlacion = df_numericas.corr(method='pearson').abs()
    redundantes = []
    columnas = correlacion.columns

    for i in range(len(columnas)):
        for j in range(i + 1, len(columnas)):
            corr_val = correlacion.iloc[i, j]
            if corr_val > umbral:
                redundantes.append((columnas[i], columnas[j], round(corr_val, 3)))

    return redundantes

In [ ]:
features_categoricas=X.select_dtypes(include=['object']).columns
features_numericas=X.select_dtypes(exclude=['object']).columns
lista=encontrar_columnas_redundantes(X[features_categoricas])
for i in lista:
  print(f'{i[0]} y {i[1]} tienen una correlacion de {i[2]}')

TIPO_SERVICIO y SERVICIO_ALTA_AGRUPADO tienen una correlacion de 0.74
INFECCIONES y INFECCION_QUIRURGICA tienen una correlacion de 0.629
PROC_7_GRUPO y PROC_8_GRUPO tienen una correlacion de 0.638
PROC_14_GRUPO y PROC_15_GRUPO tienen una correlacion de 0.712
PROC_14_GRUPO y PROC_16_GRUPO tienen una correlacion de 0.606
PROC_15_GRUPO y PROC_16_GRUPO tienen una correlacion de 0.611
PROC_16_GRUPO y PROC_17_GRUPO tienen una correlacion de 0.659
PROC_16_GRUPO y PROC_18_GRUPO tienen una correlacion de 0.67
PROC_16_GRUPO y PROC_23_GRUPO tienen una correlacion de 0.65
PROC_17_GRUPO y PROC_18_GRUPO tienen una correlacion de 0.685
PROC_17_GRUPO y PROC_23_GRUPO tienen una correlacion de 0.613
PROC_18_GRUPO y PROC_19_GRUPO tienen una correlacion de 0.731
PROC_18_GRUPO y PROC_20_GRUPO tienen una correlacion de 0.762
PROC_18_GRUPO y PROC_21_GRUPO tienen una correlacion de 0.705
PROC_18_GRUPO y PROC_22_GRUPO tienen una correlacion de 0.698
PROC_18_GRUPO y PROC_23_GRUPO tienen una correlacion de 0.755

In [ ]:
X=X.drop(columns=['PROC_14_GRUPO','PROC_19_GRUPO','PROC_20_GRUPO','PROC_21_GRUPO','PROC_22_GRUPO','PROC_23_GRUPO'])

In [ ]:
features_categoricas=X.select_dtypes(include=['object']).columns
features_numericas=X.select_dtypes(exclude=['object']).columns
lista=encontrar_columnas_redundantes(X[features_categoricas])
for i in lista:
  print(f'{i[0]} y {i[1]} tienen una correlacion de {i[2]}')

TIPO_SERVICIO y SERVICIO_ALTA_AGRUPADO tienen una correlacion de 0.74
INFECCIONES y INFECCION_QUIRURGICA tienen una correlacion de 0.629
PROC_7_GRUPO y PROC_8_GRUPO tienen una correlacion de 0.638
PROC_15_GRUPO y PROC_16_GRUPO tienen una correlacion de 0.611
PROC_16_GRUPO y PROC_17_GRUPO tienen una correlacion de 0.659
PROC_16_GRUPO y PROC_18_GRUPO tienen una correlacion de 0.67
PROC_17_GRUPO y PROC_18_GRUPO tienen una correlacion de 0.685


In [ ]:
X=X.drop(columns=['PROC_18_GRUPO','PROC_17_GRUPO','SERVICIO_ALTA_AGRUPADO'])

In [ ]:
X=X.drop(columns=['PROC_15_GRUPO','PROC_16_GRUPO'])

In [ ]:
X[features_numericas] = X[features_numericas].astype(str)
lista=encontrar_columnas_redundantes(X[features_numericas])
for i in lista:
  print(f'{i[0]} y {i[1]} tienen una correlacion de {i[2]}')

##**Analisis Target**

In [ ]:
print(y.value_counts())

GRD_CODIGO
44173     732
114122    691
54123     631
44163     578
114123    433
         ... 
134162      1
214172      1
11131       1
44111       1
194192      1
Name: count, Length: 608, dtype: int64


In [ ]:
print(y.value_counts()[y.value_counts()>200])

GRD_CODIGO
44173     732
114122    691
54123     631
44163     578
114123    433
14221     363
44162     354
114102    291
114131    286
174133    285
44172     264
81801     258
54122     252
114121    246
14263     242
54152     223
54153     208
44153     205
44181     203
44161     203
Name: count, dtype: int64


In [ ]:
print(y.value_counts()[y.value_counts()<=200])

GRD_CODIGO
64181     199
174132    198
114103    192
104102    187
81701     183
         ... 
134162      1
214172      1
11131       1
44111       1
194192      1
Name: count, Length: 588, dtype: int64


In [ ]:
y = pd.Series(y)
nuevos_GRD = y.astype(str).str.zfill(6)
y = pd.DataFrame({
    'GRD_ORIGINAL': nuevos_GRD,
})


In [ ]:
print(y.GRD_ORIGINAL.value_counts()[y.GRD_ORIGINAL.value_counts()>73])

GRD_ORIGINAL
044173    732
114122    691
054123    631
044163    578
114123    433
         ... 
064113     75
111201     74
114163     74
031202     74
084163     74
Name: count, Length: 106, dtype: int64


In [ ]:
# Paso 1: Calcular la frecuencia de cada clase
frecuencias = y.GRD_ORIGINAL.value_counts()

# Paso 2: Identificar clases frecuentes (>250)
clases_frecuentes = frecuencias[frecuencias > 73].index

# Paso 3: Crear la nueva variable con las clases raras agrupadas en "OTROS"
y['CODIGOS_SOLOS'] = y.GRD_ORIGINAL.apply(lambda x: x if x in clases_frecuentes else 'OTROS')

# Opcional: verificar resultado
print(y['CODIGOS_SOLOS'].value_counts())


CODIGOS_SOLOS
OTROS     10433
044173      732
114122      691
054123      631
044163      578
          ...  
064113       75
084163       74
031202       74
114163       74
111201       74
Name: count, Length: 107, dtype: int64


In [ ]:
y['GRD_GRUPO_2DIG'] = y['GRD_ORIGINAL'].str[:3]
conteo = y['GRD_ORIGINAL'].value_counts()
def agrupar_grd(codigo):
    if conteo[codigo] <= 100:
        return codigo[:3]
    else:
        return codigo

y['GRD_AGRUPADO'] = y['GRD_ORIGINAL'].apply(agrupar_grd)

In [ ]:
y.GRD_AGRUPADO.unique()

array(['014', '184', '054123', '074133', '081', '051', '044213', '071',
       '174133', '011', '174132', '044172', '061', '114103', '014242',
       '044162', '114', '214123', '051203', '214', '031203', '044153',
       '064173', '111', '064', '044163', '164', '074', '031', '014263',
       '014222', '064183', '044173', '114123', '054', '114122', '034',
       '044', '051153', '084', '074143', '044152', '041', '074103',
       '064172', '074113', '054122', '161', '174', '091', '094122',
       '034131', '064103', '054152', '214122', '064181', '054193',
       '081602', '194', '054202', '094', '021', '054153', '081203',
       '044143', '114121', '014221', '054203', '061203', '051202',
       '104102', '014241', '224', '121', '081202', '064182', '081601',
       '014262', '081701', '114102', '104103', '024', '041203', '131',
       '184133', '124', '071143', '101', '064112', '064111', '014231',
       '071142', '104', '044171', '094121', '134', '074142', '014232',
       '114132', '064

In [ ]:
conteo_2 = y['GRD_AGRUPADO'].value_counts()
def agrupar_target(codigo):
  if conteo_2[codigo] <= 100:
    return '0'
  else:
    return codigo
y['GRD_AGRUPADO'] = y['GRD_AGRUPADO'].apply(agrupar_target)


In [ ]:
y.GRD_AGRUPADO.value_counts()

,count
GRD_AGRUPADO,
054,1077
014,1041
081,864
084,835
064,736
...,...
094122,105
031201,104
014242,103


In [ ]:
y.GRD_AGRUPADO.unique()

array(['014', '184', '054123', '074133', '081', '051', '044213', '071',
       '174133', '011', '174132', '044172', '061', '114103', '014242',
       '044162', '114', '214123', '051203', '214', '031203', '044153',
       '064173', '111', '064', '044163', '164', '074', '031', '014263',
       '014222', '064183', '044173', '114123', '054', '114122', '034',
       '044', '051153', '084', '074143', '044152', '041', '074103',
       '064172', '074113', '054122', '161', '174', '091', '094122',
       '034131', '064103', '054152', '214122', '064181', '054193',
       '081602', '194', '054202', '094', '0', '054153', '081203',
       '044143', '114121', '014221', '054203', '061203', '051202',
       '104102', '014241', '121', '081202', '064182', '081601', '014262',
       '081701', '114102', '104103', '041203', '131', '184133', '124',
       '071143', '064112', '064111', '014231', '071142', '104', '044171',
       '094121', '074142', '014232', '114132', '064171', '081201',
       '031201', '054

In [ ]:
orden_grd = [
    '0', '011', '014', '014221', '014222', '014231', '014232', '014241', '014242', '014262', '014263',
    '031', '031201', '031203',
    '034', '034131',
    '041', '041203',
    '044', '044143', '044152', '044153', '044161', '044162', '044163', '044171', '044172', '044173', '044181', '044213',
    '051', '051153', '051202', '051203',
    '054', '054122', '054123', '054151', '054152', '054153', '054193', '054202', '054203',
    '061', '061131', '061203',
    '064', '064103', '064111', '064112', '064171', '064172', '064173', '064181', '064182', '064183',
    '071', '071141', '071142', '071143',
    '074', '074103', '074113', '074133', '074142', '074143',
    '081', '081201', '081202', '081203', '081601', '081602', '081701', '081801',
    '084',
    '091', '091301',
    '094', '094121', '094122',
    '104', '104102', '104103',
    '111', '111401',
    '114', '114102', '114103', '114121', '114122', '114123', '114131', '114132',
    '121',
    '124',
    '131',
    '161',
    '164',
    '174', '174132', '174133',
    '184', '184133',
    '194',
    '214', '214122', '214123'
]


In [ ]:
ordinal_grd = {code: i for i, code in enumerate(orden_grd)}
y['GRD_AGRUPADO_2'] = y['GRD_AGRUPADO'].map(ordinal_grd)

In [ ]:
y.GRD_AGRUPADO.unique()

array(['014', '184', '054123', '074133', '081', '051', '044213', '071',
       '174133', '011', '174132', '044172', '061', '114103', '014242',
       '044162', '114', '214123', '051203', '214', '031203', '044153',
       '064173', '111', '064', '044163', '164', '074', '031', '014263',
       '014222', '064183', '044173', '114123', '054', '114122', '034',
       '044', '051153', '084', '074143', '044152', '041', '074103',
       '064172', '074113', '054122', '161', '174', '091', '094122',
       '034131', '064103', '054152', '214122', '064181', '054193',
       '081602', '194', '054202', '094', '0', '054153', '081203',
       '044143', '114121', '014221', '054203', '061203', '051202',
       '104102', '014241', '121', '081202', '064182', '081601', '014262',
       '081701', '114102', '104103', '041203', '131', '184133', '124',
       '071143', '064112', '064111', '014231', '071142', '104', '044171',
       '094121', '074142', '014232', '114132', '064171', '081201',
       '031201', '054

Finalmente, se generaron múltiples agrupaciones de la variable Y, debido al notable desbalance entre sus clases originales. Estas agrupaciones, también denominadas targets, fueron diseñadas con el propósito de explorar diferentes formas de representar la variable de salida, permitiendo una mejor manejabilidad del desbalance. Cada una de estas versiones de Y fue evaluada estadísticamente para identificar cuál ofrecía una distribución más equilibrada y cuál facilitaba un mejor rendimiento en el análisis posterior, asegurando así que los modelos trabajen con datos más representativos y confiables.

In [ ]:
X.to_csv('X.csv', index=False)
y.to_csv('y.csv', index=False)